<a href="https://colab.research.google.com/github/workingbetter/ITNPBD5_Dissertation/blob/main/chapter_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 4.0 MB/s eta 0:00:00


In [ ]:
import random
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms

# Load charging rate data
charging_rate_df = pd.read_excel('/content/drive/MyDrive/ITNPBD5/dataset/charging_curve_data.xlsx')

# Load charging station location data
station_df = pd.read_excel('/content/drive/MyDrive/ITNPBD5/dataset/charging_station_location_data.xlsx')

# Convert the charging_rate_df data to numpy arrays for the interpolation function
minutes = charging_rate_df['minutes'].values
soc = charging_rate_df['state_of_charge'].values / 100  #  state of charge is given in percentages

# Assigning given values
initial_soc = 0.4  # initial state of charge
total_distance = 500  # total travel distance in miles
average_speed = 50  # average speed in mph (redundant not used)
discharge_rate = 0.004  # discharge rate in SOC per mile
min_soc = 0.09  # minimum SOC

def interpolate_charge_time(start_soc, end_soc):

    if start_soc > end_soc or start_soc < min(soc) or end_soc > max(soc):
        return 1e6  # Assign a high penalty if charging to the desired SOC is impossible

    start_idx = np.searchsorted(soc, start_soc, side='left')
    if soc[start_idx] > start_soc and start_idx > 0:
        start_idx -= 1

    end_idx = np.searchsorted(soc, end_soc, side='left')
    if soc[end_idx] < end_soc and end_idx < len(soc) - 1:
        end_idx += 1

    if start_idx == end_idx:
        return 0  # Start and end SOC are the same

    start_min = minutes[start_idx]
    end_min = minutes[end_idx]
    start_soc_val = soc[start_idx]
    end_soc_val = soc[end_idx]

    # Linear interpolation formula: y = y1 + ((y2 - y1) / (x2 - x1)) * (x - x1)

    # Time interpolation at the start and end SOC
    start_time = start_min + ((minutes[start_idx+1] - start_min) / (soc[start_idx+1] - start_soc_val)) * (start_soc - start_soc_val)
    end_time = end_min + ((minutes[end_idx-1] - end_min) / (soc[end_idx-1] - end_soc_val)) * (end_soc - end_soc_val)

    return (end_time - start_time)



# Define the fitness and individual classes
if "FitnessMin" in creator.__dict__:
    del creator.__dict__["FitnessMin"]
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))

if "Individual" in creator.__dict__:
    del creator.__dict__["Individual"]
creator.create('Individual', list, fitness=creator.FitnessMin, charging_times=list(), charging_stations=list(), socs_before_charging=list(), socs_after_charging=list())

toolbox = base.Toolbox()

def initialize_individual():
    individual = creator.Individual()
    for _ in range(len(station_df)):
        decision = toolbox.charging_decision()
        individual.append(decision)
    return individual

toolbox.register('individual', initialize_individual)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

toolbox.register('charging_decision', random.randint, 0, 1)

def evaluate(individual):
    total_time = 0
    soc = initial_soc
    previous_location = 0
    charging_times = []
    charging_stations = []
    socs_before_charging = []
    socs_after_charging = []
    stop_penalty = 10  # time penalty for each stop in minutes, adjust this value as needed

    for i, decision in enumerate(individual):
        stop = decision

        # Driving time to next station
        distance = station_df.loc[i, 'location(mile)'] - previous_location
        driving_time = (distance / average_speed) * 60  # Convert hours to minutes
        total_time += driving_time

        # Update SOC and location
        soc -= distance * discharge_rate
        previous_location = station_df.loc[i, 'location(mile)']

        # Check if the SOC is enough to reach the station
        if soc < min_soc:
            return float('inf'),  # Return very high fitness value

        if stop:
            # Generate a new charging target between current SOC and 1
            new_charging_target = round(random.uniform(soc, 1), 2)

            # Cannot charge if the new target SOC is less than the current SOC
            if new_charging_target <= soc:
                return float('inf'),  # Return very high fitness value

            # Record SOC before charging
            socs_before_charging.append(soc)

            charging_time = interpolate_charge_time(soc, new_charging_target)
            charging_time += stop_penalty  # Add stop penalty to charging time
            total_time += charging_time  # Add the charging time (including the stop penalty) to the total time

            charging_times.append(charging_time)  # Record charging time (including the stop penalty)
            charging_stations.append(i)

            soc = new_charging_target  # Update SOC

            # Record SOC after charging
            socs_after_charging.append(soc)

    individual.charging_times = charging_times
    individual.charging_stations = charging_stations
    individual.socs_before_charging = socs_before_charging
    individual.socs_after_charging = socs_after_charging

    return total_time,



# Mutation
def mutate(individual):
    soc = initial_soc
    for i in range(len(individual)):
        decision = individual[i]

        # Mutate decision with a 5% probability
        if random.random() < 0.05:
            new_decision = 1 - decision
            individual[i] = new_decision

    return individual,

toolbox.register('mutate', mutate)

def mate(ind1, ind2):
    for i in range(len(ind1)):
        if random.random() < 0.5:  # 50% probability to swap each decision and amount
            ind1[i], ind2[i] = ind2[i], ind1[i]
    return ind1, ind2

toolbox.register('mate', mate)


# Define the genetic operators
toolbox.register('evaluate', evaluate)
toolbox.register('select', tools.selTournament, tournsize=3)

# Define the main function
def main():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('Avg', np.mean)
    stats.register('Min', np.min)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=150, stats=stats, halloffame=hof, verbose=True)

    best_individual = hof[0]
    total_travel_time_mins = best_individual.fitness.values[0]
    total_travel_time = f'{int(total_travel_time_mins // 60)} hours {int(total_travel_time_mins % 60)} minutes'

    # Print the total travel time
    print(f'Total travel time: {total_travel_time}')

    # Print charging times and stations
    print('Charging times: ', best_individual.charging_times)
    print('Charging stations: ', best_individual.charging_stations)

    return pop, log, hof


# Run the GA
pop, log, hof = main()

# Extract the best individual
best_individual = hof[0]

# Prepare data for DataFrame
data = {
    'Charging Stations': best_individual.charging_stations,
    'Charging Time(minute)': best_individual.charging_times,
    'SOCs Before Charging': best_individual.socs_before_charging,
    'SOCs After Charging': best_individual.socs_after_charging
}

# Create DataFrame
output_df = pd.DataFrame(data)
# Extract the distance information
distances = [station_df.loc[i, 'location(mile)'] for i in best_individual.charging_stations]

# Add the distance information to the DataFrame
output_df['Distance (mile)'] = distances

display(output_df)

gen	nevals	Avg	Min   
0  	300   	inf	1338.1
1  	188   	inf	1338.1
2  	182   	inf	1396.7
3  	177   	inf	1381.87
4  	184   	inf	1329.97
5  	169   	inf	1325.27
6  	177   	inf	1283.37
7  	188   	inf	1286.4 
8  	193   	inf	1211.8 
9  	174   	inf	1211.8 
10 	191   	inf	1160.17
11 	199   	inf	1022.13
12 	184   	inf	1136.53
13 	161   	inf	1112.83
14 	176   	inf	1095.8 
15 	185   	inf	1025.3 
16 	171   	inf	1065.13
17 	186   	inf	981.967
18 	186   	inf	922.067
19 	176   	inf	913.867
20 	193   	inf	922.1  
21 	168   	inf	909.467
22 	197   	inf	899.267
23 	171   	inf	869.7  
24 	182   	inf	869.7  
25 	174   	inf	839.233
26 	166   	inf	839.233
27 	177   	inf	807.1  
28 	157   	inf	807.1  
29 	190   	inf	799.967
30 	194   	inf	789.067
31 	186   	inf	771.4  
32 	173   	inf	771.4  
33 	159   	inf	759.733
34 	174   	inf	766.1  
35 	161   	inf	752.633
36 	187   	inf	739.533
37 	185   	inf	739.533
38 	195   	inf	739.533
39 	180   	inf	739.533
40 	189   	inf	739.533
41 	177   	inf	739.533
42 	191   	inf	

,Charging Stations,Charging Time(minute),SOCs Before Charging,SOCs After Charging,Distance (mile)
0,10,39.300000,0.224,0.85,44
1,34,21.700000,0.466,0.74,140
2,61,26.733333,0.308,0.72,248
3,82,27.366667,0.384,0.78,332


In [ ]:
# Extract the distance information
distances = [station_df.loc[i, 'location(mile)'] for i in best_individual.charging_stations]

# Add the distance information to the DataFrame
output_df['Distance (mile)'] = distances

display(output_df)
# Save the DataFrame to an Excel file
# output_df.to_excel('/content/drive/MyDrive/ITNPBD5/dataset/output.xlsx', index=False)


NameError: ignored